# Subtask goal post processing

In [1]:
import pandas as pd
from difflib import get_close_matches

cc_website_goal_table_path= "/Users/cheng/Documents/GitHub/subtask_benchmark/src/subtask_benchmark/synthetic_data_gen/data/cc_website_data.csv"
commercial_website_goal_table_path= "/Users/cheng/Documents/GitHub/subtask_benchmark/src/subtask_benchmark/synthetic_data_gen/data/commercial_website_data.csv"
ui_website_goal_table_path= "/Users/cheng/Documents/GitHub/subtask_benchmark/src/subtask_benchmark/synthetic_data_gen/data/ui_website_data.csv"

preflight_parquet_path = "/Users/cheng/Documents/GitHub/subtask_benchmark/src/subtask_benchmark/synthetic_data_gen/data/goal_table_combined_preflight.parquet"
full_parquet_path = "/Users/cheng/Documents/GitHub/subtask_benchmark/src/subtask_benchmark/synthetic_data_gen/data/goal_table_combined_full.parquet"

In [2]:
cc_website_goal_table = pd.read_csv(cc_website_goal_table_path)
commercial_website_goal_table = pd.read_csv(commercial_website_goal_table_path)
ui_website_goal_table = pd.read_csv(ui_website_goal_table_path)


In [3]:
TASK_TYPES = [
    "dropdown_option_selection",
    "table_manipulation",
    "menu_navigation",
    "computation",
    "search_and_autocomplete",
    "icon_recognition",
    "data_extraction",
    "form_filling",
    "datepicker",
    "sheet_editing",
    "document_editing",
    "pagination",
    "dialog_boxes",
    "generic_grounding",
    "list_navigation",
    "drag_and_drop_interaction",
]

cc_website_goal_table["website_type"] = "cc"
commercial_website_goal_table["website_type"] = "commercial"
ui_website_goal_table["website_type"] = "ui"

goal_table = pd.concat([cc_website_goal_table, commercial_website_goal_table, ui_website_goal_table])
goal_table = goal_table.drop("data_path", axis=1)
goal_table["subtask_type"] = goal_table["subtask_type"].apply(lambda x: x.split(",")[0])
goal_table["subtask_type"] = goal_table["subtask_type"].apply(lambda x: x if x in TASK_TYPES else get_close_matches(x, TASK_TYPES, n=1, cutoff=0.0)[0])
goal_table = goal_table.rename(columns={"subtask_goal": "goal", "start_url": "base_url"})


goal_table = goal_table.sample(frac=1)

goal_table_preflight, goal_table_full = goal_table.iloc[:100], goal_table.iloc[100:]

goal_table_preflight.to_parquet(preflight_parquet_path, index=False)
goal_table_full.to_parquet(full_parquet_path, index=False)

# goal_table.sample(10)

In [ ]:
goal_table_preflight = pd.read_parquet(preflight_parquet_path)
goal_table_full = pd.read_parquet(full_parquet_path)

print("subtask type counts:")
print(goal_table_preflight["subtask_type"].value_counts())
print(goal_table_full["subtask_type"].value_counts())

In [ ]:
print("website type counts:")
print(goal_table_preflight["website_type"].value_counts())
print(goal_table_full["website_type"].value_counts())